# Día 4: Principios SOLID

## Descripción General

Los **principios SOLID** son cinco principios de diseño orientado a objetos que ayudan a crear software más mantenible, flexible y escalable. Fueron introducidos por Robert C. Martin y son fundamentales en el desarrollo de software profesional.

SOLID es un acrónimo de:
- **S**ingle Responsibility Principle (SRP)
- **O**pen/Closed Principle (OCP)
- **L**iskov Substitution Principle (LSP)
- **I**nterface Segregation Principle (ISP)
- **D**ependency Inversion Principle (DIP)

**Conexión con Data/IA**: En proyectos de ML, los principios SOLID son críticos para crear pipelines mantenibles, modelos intercambiables, y sistemas que escalan. Sin SOLID, tu código de ML se convierte en un monolito imposible de mantener.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Comprender los cinco principios SOLID y por qué existen
2. Identificar violaciones de SOLID en código real
3. Aplicar cada principio SOLID en código Python
4. Refactorizar código para cumplir con SOLID
5. Desarrollar intuición para saber cuándo aplicar cada principio
6. Diseñar sistemas de ML/Data siguiendo SOLID

## 1. Single Responsibility Principle (SRP)

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Tienes una clase `ModelTrainer` que entrena modelos, guarda en S3, envía emails, loggea métricas, y actualiza base de datos. **5 responsabilidades en una clase** 💥 Cambiar logging = **riesgo de romper entrenamiento** 💥 Testing = **mockear 5 dependencias** 💥 Reutilizar solo el entrenamiento = **imposible** 💥

Sin SRP: Clase hace TODO. Cambio en una parte = **riesgo en todas las partes**. Bug en email = **entrenamiento se cae**. Imposible testear en aislamiento.

Con SRP: Cada clase hace UNA cosa. `ModelTrainer` solo entrena. `ModelStorage` solo guarda. `NotificationService` solo notifica. **Cambios aislados** ✅ **Testing simple** ✅ **Reutilización fácil** ✅

**Ejemplo concreto para juniors**:

Clase `User` que valida datos, guarda en DB, envía email de bienvenida, y genera reporte. **4 razones para cambiar** 💥 Cambiar formato de email = **modificar clase User** 💥 Cambiar DB = **modificar clase User** 💥

Con SRP: `UserValidator`, `UserRepository`, `EmailService`, `ReportGenerator`. Cada uno hace UNA cosa. Cambiar email = **solo modificar EmailService** ✅

**Consecuencias de NO usarlo**:
- **Acoplamiento alto** → cambio en una parte afecta otras ($30K en bugs)
- **Testing complejo** → mockear múltiples dependencias (3x tiempo)
- **Reutilización imposible** → no puedes usar solo una parte
- **Bugs multiplicados** → cambio en logging rompe entrenamiento
- **Merge conflicts** → 5 devs modificando la misma clase

### 📚 El Concepto

**Definición técnica**:

**Single Responsibility Principle (SRP)**: Una clase debe tener **una sola razón para cambiar**. Cada clase debe tener **una única responsabilidad** bien definida. Si una clase hace múltiples cosas, debe dividirse en clases más pequeñas.

**Cómo funciona internamente**:
1. Identificas todas las responsabilidades de una clase
2. Si hay más de una → viola SRP
3. Extraes cada responsabilidad a su propia clase
4. Cada clase ahora tiene una sola razón para cambiar
5. Clases se comunican mediante interfaces claras
6. Cambios en una responsabilidad no afectan otras

**Terminología clave**:
- **Responsabilidad**: Razón para cambiar (feature, lógica de negocio)
- **Cohesión**: Qué tan relacionadas están las funciones de una clase (alta cohesión = bueno)
- **Acoplamiento**: Qué tan dependientes son las clases entre sí (bajo acoplamiento = bueno)
- **Separación de concerns**: Dividir sistema en partes distintas con responsabilidades únicas

### ❌ Ejemplo Incorrecto: Múltiples Responsabilidades

**Código**:

In [ ]:
# ❌ BAD: Multiple responsibilities in one class
import json
from typing import Dict, List

class ModelTrainer:
    """Model trainer - DOES EVERYTHING! Violates SRP."""
    
    def __init__(self, model_type: str):
        self.model_type = model_type
        self.model = None
    
    def train(self, X: List, y: List) -> None:
        """Train model - Responsibility 1."""
        print(f"Training {self.model_type}...")
        self.model = "trained_model"
    
    def save_to_s3(self, bucket: str, key: str) -> None:
        """Save to S3 - Responsibility 2."""
        print(f"Saving to s3://{bucket}/{key}")
    
    def send_email(self, recipient: str) -> None:
        """Send notification email - Responsibility 3."""
        print(f"Sending email to {recipient}")
    
    def log_metrics(self, metrics: Dict) -> None:
        """Log metrics - Responsibility 4."""
        print(f"Logging metrics: {json.dumps(metrics)}")
    
    def update_database(self, model_id: str) -> None:
        """Update DB - Responsibility 5."""
        print(f"Updating DB for model {model_id}")

# Usage - tightly coupled, hard to test
trainer = ModelTrainer("RandomForest")
trainer.train([1, 2, 3], [0, 1, 0])
trainer.save_to_s3("my-bucket", "model.pkl")
trainer.send_email("team@company.com")
trainer.log_metrics({"accuracy": 0.95})
trainer.update_database("model_123")

**Problemas**:
- **5 responsabilidades** → training, storage, email, logging, database
- **5 razones para cambiar** → cambio en cualquiera modifica la clase
- **Testing pesadilla** → mockear S3, email, logger, DB para testear training
- **Reutilización imposible** → no puedes usar solo el training
- **Acoplamiento alto** → bug en email puede romper training
- **Merge conflicts** → múltiples devs modificando la misma clase

### ✅ Ejemplo Correcto: Una Responsabilidad por Clase

**Código**:

In [ ]:
# ✅ GOOD: Single responsibility per class
import json
from typing import Dict, List, Any

class ModelTrainer:
    """Trains ML models - ONLY training responsibility."""
    
    def __init__(self, model_type: str) -> None:
        self.model_type = model_type
        self.model: Any = None
    
    def train(self, X: List, y: List) -> Any:
        """
        Train the model.
        
        :param X: Training features
        :type X: List
        :param y: Training labels
        :type y: List
        :return: Trained model
        :rtype: Any
        """
        print(f"Training {self.model_type}...")
        self.model = "trained_model"
        return self.model

class ModelStorage:
    """Stores models - ONLY storage responsibility."""
    
    def save_to_s3(self, model: Any, bucket: str, key: str) -> str:
        """
        Save model to S3.
        
        :param model: Model to save
        :type model: Any
        :param bucket: S3 bucket name
        :type bucket: str
        :param key: S3 key
        :type key: str
        :return: S3 URL
        :rtype: str
        """
        url = f"s3://{bucket}/{key}"
        print(f"Saving to {url}")
        return url

class NotificationService:
    """Sends notifications - ONLY notification responsibility."""
    
    def send_email(self, recipient: str, subject: str, body: str) -> None:
        """
        Send email notification.
        
        :param recipient: Email recipient
        :type recipient: str
        :param subject: Email subject
        :type subject: str
        :param body: Email body
        :type body: str
        """
        print(f"Sending email to {recipient}: {subject}")

class MetricsLogger:
    """Logs metrics - ONLY logging responsibility."""
    
    def log(self, metrics: Dict) -> None:
        """
        Log metrics.
        
        :param metrics: Metrics to log
        :type metrics: Dict
        """
        print(f"Logging metrics: {json.dumps(metrics)}")

class ModelRepository:
    """Manages model metadata - ONLY database responsibility."""
    
    def update(self, model_id: str, metadata: Dict) -> None:
        """
        Update model metadata in database.
        
        :param model_id: Model identifier
        :type model_id: str
        :param metadata: Model metadata
        :type metadata: Dict
        """
        print(f"Updating DB for model {model_id}")

# Usage - loosely coupled, easy to test
trainer = ModelTrainer("RandomForest")
storage = ModelStorage()
notifier = NotificationService()
logger = MetricsLogger()
repo = ModelRepository()

# Each component does ONE thing
model = trainer.train([1, 2, 3], [0, 1, 0])
url = storage.save_to_s3(model, "my-bucket", "model.pkl")
notifier.send_email("team@company.com", "Training Complete", f"Model at {url}")
logger.log({"accuracy": 0.95})
repo.update("model_123", {"url": url, "accuracy": 0.95})

**Ventajas**:
- **1 responsabilidad por clase** → cada clase hace UNA cosa
- **1 razón para cambiar** → cambio en storage no afecta training
- **Testing simple** → testear training sin mockear S3, email, DB
- **Reutilización fácil** → usar `ModelStorage` en otros contextos
- **Acoplamiento bajo** → bug en email no afecta training
- **Sin merge conflicts** → cada dev trabaja en su clase

### 📊 Comparación Lado a Lado

| Aspecto | Sin SRP | Con SRP |
|---------|---------|----------|
| Responsabilidades | 5 en 1 clase | 1 por clase |
| Razones para cambiar | 5 | 1 |
| Testing | Complejo (5 mocks) | Simple (1 mock) |
| Reutilización | Imposible | Fácil |
| Acoplamiento | Alto | Bajo |
| Merge conflicts | Frecuentes | Raros |

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Una responsabilidad = una razón para cambiar**
2. **Alta cohesión** → métodos de la clase están relacionados
3. **Bajo acoplamiento** → clases independientes entre sí
4. **Testing simple** → testear una cosa a la vez
5. **Reutilización** → clases pequeñas son reutilizables

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Cuántas razones tiene esta clase para cambiar?"
  - 1 razón → cumple SRP ✅
  - 2+ razones → viola SRP, dividir ❌

- **Pregúntate**: "¿Puedo describir esta clase en una frase sin usar 'y'?"
  - SÍ → cumple SRP ✅
  - NO ("entrena Y guarda Y notifica") → viola SRP ❌

- **Pregúntate**: "¿Necesito mockear múltiples cosas para testear esta clase?"
  - NO → cumple SRP ✅
  - SÍ → viola SRP, dividir ❌

**Cuándo usar / NO usar**:
- ✅ **Usar SRP cuando**:
  - Clase hace múltiples cosas no relacionadas
  - Testing requiere múltiples mocks
  - Múltiples devs modifican la misma clase
  - Quieres reutilizar solo parte de la funcionalidad
- ❌ **NO sobre-aplicar cuando**:
  - Clase es trivial (getter/setter)
  - Dividir crea complejidad innecesaria
  - Responsabilidades están íntimamente relacionadas

**Referencia oficial:** [SOLID Principles - SRP](https://en.wikipedia.org/wiki/Single-responsibility_principle)

## 2. Open/Closed Principle (OCP)

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Tienes función `calculate_metrics()` con if/elif para cada métrica (accuracy, precision, recall, F1). Añadir nueva métrica = **modificar función existente** 💥 Riesgo de **romper métricas existentes** 💥 Testing = **re-testear todo** 💥 10 métricas = **función de 200 líneas** 💥

Sin OCP: Añadir funcionalidad = modificar código existente. **Alto riesgo** 💥 Cada cambio puede introducir bugs en código que funcionaba.

Con OCP: Añadir funcionalidad = **añadir código nuevo**, no modificar existente. Cada métrica es una clase. Nueva métrica = **nueva clase** ✅ **Cero riesgo** en métricas existentes ✅

**Ejemplo concreto para juniors**:

Sistema de descuentos con if/elif para cada tipo (estudiante, senior, empleado). Nuevo tipo = **modificar función** 💥 Riesgo de romper descuentos existentes 💥

Con OCP: Cada descuento es una clase que implementa interfaz `Discount`. Nuevo descuento = **nueva clase** ✅ Sin tocar código existente ✅

**Consecuencias de NO usarlo**:
- **Bugs en código estable** → modificar código que funcionaba ($40K en bugs)
- **Testing completo** → re-testear todo cada vez (2x tiempo)
- **Merge conflicts** → todos modifican la misma función
- **Funciones gigantes** → 500 líneas con 20 if/elif
- **Miedo a cambiar** → "si funciona, no lo toques"

### 📚 El Concepto

**Definición técnica**:

**Open/Closed Principle (OCP)**: Las entidades de software (clases, módulos, funciones) deben estar **abiertas para extensión** pero **cerradas para modificación**. Puedes añadir nueva funcionalidad sin cambiar código existente.

**Cómo funciona internamente**:
1. Defines interfaz o clase base abstracta
2. Implementaciones concretas heredan/implementan la interfaz
3. Código cliente usa la interfaz, no implementaciones concretas
4. Nueva funcionalidad = nueva implementación de la interfaz
5. Código existente no se modifica
6. Extensión mediante herencia/composición, no modificación

**Terminología clave**:
- **Abierto para extensión**: Puedes añadir nuevo comportamiento
- **Cerrado para modificación**: No modificas código existente
- **Abstracción**: Interfaz o clase base que define contrato
- **Polimorfismo**: Múltiples implementaciones de la misma interfaz
- **Strategy Pattern**: Patrón de diseño que implementa OCP

### ❌ Ejemplo Incorrecto: Modificar Código Existente

**Código**:

In [ ]:
# ❌ BAD: Violates OCP - must modify function to add new metric
from typing import List

def calculate_metric(y_true: List[int], y_pred: List[int], metric_type: str) -> float:
    """
    Calculate metric - VIOLATES OCP!
    Adding new metric requires modifying this function.
    """
    if metric_type == "accuracy":
        correct = sum(1 for t, p in zip(y_true, y_pred) if t == p)
        return correct / len(y_true)
    
    elif metric_type == "precision":
        tp = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 1)
        fp = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 1)
        return tp / (tp + fp) if (tp + fp) > 0 else 0.0
    
    elif metric_type == "recall":
        tp = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 1)
        fn = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 0)
        return tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    # Want to add F1? Must modify this function! 💥
    # Want to add AUC? Must modify this function! 💥
    # 10 metrics = 200 lines of if/elif! 💥
    
    else:
        raise ValueError(f"Unknown metric: {metric_type}")

# Usage - tightly coupled to metric names
y_true = [1, 0, 1, 1, 0]
y_pred = [1, 0, 0, 1, 0]

print(f"Accuracy: {calculate_metric(y_true, y_pred, 'accuracy')}")
print(f"Precision: {calculate_metric(y_true, y_pred, 'precision')}")
print(f"Recall: {calculate_metric(y_true, y_pred, 'recall')}")

**Problemas**:
- **Modificar para extender** → añadir métrica = modificar función
- **Riesgo alto** → cambio puede romper métricas existentes
- **Testing completo** → re-testear todas las métricas
- **Función gigante** → 10 métricas = 200+ líneas
- **Merge conflicts** → todos modifican la misma función
- **Acoplamiento** → código cliente conoce nombres de métricas

### ✅ Ejemplo Correcto: Extensión sin Modificación

**Código**:

In [ ]:
# ✅ GOOD: Follows OCP - extend by adding new classes
from abc import ABC, abstractmethod
from typing import List

class Metric(ABC):
    """Abstract base class for metrics - defines contract."""
    
    @abstractmethod
    def calculate(self, y_true: List[int], y_pred: List[int]) -> float:
        """
        Calculate metric.
        
        :param y_true: True labels
        :type y_true: List[int]
        :param y_pred: Predicted labels
        :type y_pred: List[int]
        :return: Metric value
        :rtype: float
        """
        pass

class Accuracy(Metric):
    """Accuracy metric implementation."""
    
    def calculate(self, y_true: List[int], y_pred: List[int]) -> float:
        """Calculate accuracy."""
        correct = sum(1 for t, p in zip(y_true, y_pred) if t == p)
        return correct / len(y_true)

class Precision(Metric):
    """Precision metric implementation."""
    
    def calculate(self, y_true: List[int], y_pred: List[int]) -> float:
        """Calculate precision."""
        tp = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 1)
        fp = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 1)
        return tp / (tp + fp) if (tp + fp) > 0 else 0.0

class Recall(Metric):
    """Recall metric implementation."""
    
    def calculate(self, y_true: List[int], y_pred: List[int]) -> float:
        """Calculate recall."""
        tp = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 1)
        fn = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 0)
        return tp / (tp + fn) if (tp + fn) > 0 else 0.0

# Want to add F1? Just create new class! No modification needed! ✅
class F1Score(Metric):
    """F1 Score metric - NEW class, no modification to existing code!"""
    
    def calculate(self, y_true: List[int], y_pred: List[int]) -> float:
        """Calculate F1 score."""
        precision = Precision().calculate(y_true, y_pred)
        recall = Recall().calculate(y_true, y_pred)
        if precision + recall == 0:
            return 0.0
        return 2 * (precision * recall) / (precision + recall)

class MetricCalculator:
    """Calculates metrics - works with any Metric implementation."""
    
    def calculate_all(self, metrics: List[Metric], y_true: List[int], y_pred: List[int]) -> dict:
        """
        Calculate all metrics.
        
        :param metrics: List of metric implementations
        :type metrics: List[Metric]
        :param y_true: True labels
        :type y_true: List[int]
        :param y_pred: Predicted labels
        :type y_pred: List[int]
        :return: Dictionary of metric results
        :rtype: dict
        """
        results = {}
        for metric in metrics:
            name = metric.__class__.__name__
            results[name] = metric.calculate(y_true, y_pred)
        return results

# Usage - loosely coupled, extensible
y_true = [1, 0, 1, 1, 0]
y_pred = [1, 0, 0, 1, 0]

calculator = MetricCalculator()
metrics = [Accuracy(), Precision(), Recall(), F1Score()]  # Easy to add new metrics!

results = calculator.calculate_all(metrics, y_true, y_pred)
for name, value in results.items():
    print(f"{name}: {value:.3f}")

**Ventajas**:
- **Extensión sin modificación** → nueva métrica = nueva clase
- **Cero riesgo** → código existente no se toca
- **Testing aislado** → testear solo la nueva métrica
- **Clases pequeñas** → cada métrica en su clase
- **Sin merge conflicts** → cada dev crea su clase
- **Polimorfismo** → `MetricCalculator` funciona con cualquier `Metric`

### 📊 Comparación Lado a Lado

| Aspecto | Sin OCP | Con OCP |
|---------|---------|----------|
| Añadir funcionalidad | Modificar código | Añadir clase |
| Riesgo | Alto (romper existente) | Cero |
| Testing | Re-testear todo | Solo nuevo código |
| Tamaño de función | 200+ líneas | 10-20 líneas |
| Merge conflicts | Frecuentes | Raros |
| Extensibilidad | Difícil | Fácil |

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Abierto para extensión** → añadir funcionalidad fácilmente
2. **Cerrado para modificación** → no tocar código que funciona
3. **Abstracción** → usar interfaces/ABC para definir contratos
4. **Polimorfismo** → múltiples implementaciones de la misma interfaz
5. **Strategy Pattern** → patrón común que implementa OCP

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Tengo que modificar código existente para añadir funcionalidad?"
  - SÍ → viola OCP, refactorizar ❌
  - NO (solo añado clase nueva) → cumple OCP ✅

- **Pregúntate**: "¿Tengo función gigante con muchos if/elif?"
  - SÍ → viola OCP, extraer a clases ❌
  - NO → cumple OCP ✅

- **Pregúntate**: "¿Puedo añadir nueva funcionalidad sin riesgo de romper existente?"
  - SÍ → cumple OCP ✅
  - NO → viola OCP ❌

**Cuándo usar / NO usar**:
- ✅ **Usar OCP cuando**:
  - Múltiples variantes de un comportamiento (métricas, modelos, estrategias)
  - Funcionalidad crece frecuentemente
  - Quieres extensibilidad sin riesgo
  - Código estable que no debe modificarse
- ❌ **NO sobre-aplicar cuando**:
  - Funcionalidad es simple y estable
  - Solo 2-3 casos (if/else simple está bien)
  - Abstracción añade complejidad innecesaria

**Referencia oficial:** [SOLID Principles - OCP](https://en.wikipedia.org/wiki/Open%E2%80%93closed_principle)

## 3. Liskov Substitution Principle (LSP)

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Tienes clase base `Model` con método `train(X, y)`. Creas `PretrainedModel` que hereda pero **lanza excepción** en `train()` porque ya está entrenado. Código que usa `Model` **explota** 💥 cuando recibe `PretrainedModel` 💥 **Herencia rota** 💥

Sin LSP: Subclases cambian comportamiento esperado. Código que funciona con clase base **falla** con subclase. **Bugs silenciosos** 💥 **Testing incompleto** 💥

Con LSP: Subclases respetan contrato de clase base. `PretrainedModel.train()` es no-op (no hace nada) pero **no lanza excepción** ✅ Código funciona con cualquier `Model` ✅

**Ejemplo concreto para juniors**:

Clase `Rectangle` con `set_width()` y `set_height()`. Creas `Square` que hereda pero **modifica ambos** cuando cambias uno (cuadrado tiene lados iguales). Código que usa `Rectangle` **falla** 💥 porque espera independencia entre width/height.

Con LSP: `Square` NO hereda de `Rectangle` (viola contrato). Ambos implementan interfaz `Shape` ✅

**Consecuencias de NO usarlo**:
- **Bugs en runtime** → código falla con subclases ($60K en bugs)
- **Testing incompleto** → tests pasan con base, fallan con subclases
- **Herencia rota** → subclases no son sustituibles
- **Código defensivo** → if/isinstance checks everywhere
- **Polimorfismo roto** → no puedes usar subclases intercambiablemente

### 📚 El Concepto

**Definición técnica**:

**Liskov Substitution Principle (LSP)**: Los objetos de una subclase deben poder **sustituir** objetos de la clase base sin alterar el comportamiento correcto del programa. Si `S` es subtipo de `T`, entonces objetos de tipo `T` pueden ser reemplazados por objetos de tipo `S` sin romper el programa.

**Cómo funciona internamente**:
1. Clase base define contrato (precondiciones, postcondiciones, invariantes)
2. Subclase hereda de clase base
3. Subclase **respeta** el contrato de la base
4. Subclase puede **fortalecer** postcondiciones (hacer más)
5. Subclase puede **debilitar** precondiciones (aceptar más)
6. Subclase **NO puede** lanzar excepciones nuevas
7. Código cliente funciona con base y todas las subclases

**Terminología clave**:
- **Sustituibilidad**: Subclase puede reemplazar clase base sin problemas
- **Contrato**: Precondiciones, postcondiciones, invariantes de la clase
- **Precondición**: Lo que debe ser cierto antes de llamar método
- **Postcondición**: Lo que debe ser cierto después de llamar método
- **Invariante**: Lo que siempre debe ser cierto para la clase

### ❌ Ejemplo Incorrecto: Subclase Rompe Contrato

**Código**:

In [ ]:
# ❌ BAD: Violates LSP - subclass breaks contract
from typing import List, Any

class Model:
    """Base model class - defines contract."""
    
    def train(self, X: List, y: List) -> None:
        """
        Train the model.
        Contract: This method should always succeed with valid data.
        """
        print("Training model...")
        self.trained = True
    
    def predict(self, X: List) -> List:
        """Make predictions."""
        if not hasattr(self, 'trained'):
            raise ValueError("Model not trained")
        return [1] * len(X)

class PretrainedModel(Model):
    """Pretrained model - VIOLATES LSP!"""
    
    def __init__(self):
        self.trained = True
    
    def train(self, X: List, y: List) -> None:
        """
        VIOLATES LSP! Raises exception instead of training.
        Breaks the contract that train() should always succeed.
        """
        raise RuntimeError("Cannot train pretrained model!")  # 💥 Breaks contract!
    
    def predict(self, X: List) -> List:
        """Make predictions."""
        return [0] * len(X)

def train_and_evaluate(model: Model, X: List, y: List) -> float:
    """
    Train and evaluate model.
    Expects ANY Model to work - but PretrainedModel breaks!
    """
    model.train(X, y)  # 💥 Fails with PretrainedModel!
    predictions = model.predict(X)
    accuracy = sum(1 for p, t in zip(predictions, y) if p == t) / len(y)
    return accuracy

# Usage - works with Model
X = [[1, 2], [3, 4]]
y = [0, 1]

model = Model()
print(f"Model accuracy: {train_and_evaluate(model, X, y)}")

# But fails with PretrainedModel - VIOLATES LSP!
pretrained = PretrainedModel()
try:
    print(f"Pretrained accuracy: {train_and_evaluate(pretrained, X, y)}")
except RuntimeError as e:
    print(f"\n💥 LSP violation! {e}")
    print("PretrainedModel cannot substitute Model!")

**Problemas**:
- **Rompe contrato** → `train()` debe funcionar, pero lanza excepción
- **No sustituible** → `PretrainedModel` no puede reemplazar `Model`
- **Bugs en runtime** → código que funciona con `Model` falla con subclase
- **Testing incompleto** → tests con `Model` pasan, con subclase fallan
- **Código defensivo** → necesitas `isinstance` checks
- **Polimorfismo roto** → no puedes usar subclases intercambiablemente

### ✅ Ejemplo Correcto: Subclase Respeta Contrato

**Código**:

In [ ]:
# ✅ GOOD: Follows LSP - subclass respects contract
from typing import List, Any

class Model:
    """Base model class - defines contract."""
    
    def train(self, X: List, y: List) -> None:
        """
        Train the model.
        Contract: This method should always succeed with valid data.
        
        :param X: Training features
        :type X: List
        :param y: Training labels
        :type y: List
        """
        print("Training model...")
        self.trained = True
    
    def predict(self, X: List) -> List:
        """
        Make predictions.
        
        :param X: Features
        :type X: List
        :return: Predictions
        :rtype: List
        """
        if not hasattr(self, 'trained'):
            raise ValueError("Model not trained")
        return [1] * len(X)

class PretrainedModel(Model):
    """Pretrained model - RESPECTS LSP!"""
    
    def __init__(self):
        self.trained = True
    
    def train(self, X: List, y: List) -> None:
        """
        RESPECTS LSP! No-op instead of exception.
        Contract is respected: method succeeds, just does nothing.
        """
        print("Model already trained, skipping...")
        # No exception! Just a no-op. Contract respected! ✅
    
    def predict(self, X: List) -> List:
        """Make predictions."""
        return [0] * len(X)

class TrainableModel(Model):
    """Trainable model - also respects LSP."""
    
    def train(self, X: List, y: List) -> None:
        """Train the model."""
        print("Training trainable model...")
        self.trained = True
        self.X_train = X
        self.y_train = y
    
    def predict(self, X: List) -> List:
        """Make predictions."""
        if not hasattr(self, 'trained'):
            raise ValueError("Model not trained")
        return [1] * len(X)

def train_and_evaluate(model: Model, X: List, y: List) -> float:
    """
    Train and evaluate model.
    Works with ANY Model subclass - LSP respected!
    
    :param model: Model instance
    :type model: Model
    :param X: Features
    :type X: List
    :param y: Labels
    :type y: List
    :return: Accuracy
    :rtype: float
    """
    model.train(X, y)  # ✅ Works with all subclasses!
    predictions = model.predict(X)
    accuracy = sum(1 for p, t in zip(predictions, y) if p == t) / len(y)
    return accuracy

# Usage - works with all subclasses!
X = [[1, 2], [3, 4]]
y = [0, 1]

model = Model()
print(f"Model accuracy: {train_and_evaluate(model, X, y)}")

pretrained = PretrainedModel()
print(f"Pretrained accuracy: {train_and_evaluate(pretrained, X, y)}")

trainable = TrainableModel()
print(f"Trainable accuracy: {train_and_evaluate(trainable, X, y)}")

print("\n✅ All subclasses are substitutable! LSP respected!")

**Ventajas**:
- **Respeta contrato** → `train()` siempre funciona (no-op es válido)
- **Sustituible** → todas las subclases pueden reemplazar base
- **Sin bugs** → código funciona con base y todas las subclases
- **Testing completo** → tests con base funcionan con subclases
- **Sin código defensivo** → no necesitas `isinstance` checks
- **Polimorfismo funciona** → subclases son intercambiables

### 📊 Comparación Lado a Lado

| Aspecto | Sin LSP | Con LSP |
|---------|---------|----------|
| Contrato | Roto | Respetado |
| Sustituibilidad | No | Sí |
| Bugs en runtime | Frecuentes | Ninguno |
| Testing | Incompleto | Completo |
| Código defensivo | Necesario | Innecesario |
| Polimorfismo | Roto | Funciona |

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Sustituibilidad** → subclase debe poder reemplazar base sin problemas
2. **Respetar contrato** → precondiciones, postcondiciones, invariantes
3. **No lanzar excepciones nuevas** → subclase no puede ser más restrictiva
4. **No-op > excepción** → si no aplica, no hacer nada es mejor que fallar
5. **Herencia es-un** → solo hereda si realmente "es un" tipo de la base

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Puedo reemplazar clase base con subclase sin romper código?"
  - SÍ → cumple LSP ✅
  - NO → viola LSP, refactorizar ❌

- **Pregúntate**: "¿La subclase lanza excepciones que la base no lanza?"
  - SÍ → viola LSP ❌
  - NO → cumple LSP ✅

- **Pregúntate**: "¿La subclase cambia comportamiento esperado de la base?"
  - SÍ → viola LSP, considerar composición ❌
  - NO → cumple LSP ✅

**Cuándo usar / NO usar**:
- ✅ **Usar herencia cuando**:
  - Subclase realmente "es un" tipo de la base
  - Subclase respeta contrato de la base
  - Subclase puede sustituir base sin problemas
  - Relación es-un es clara y natural
- ❌ **NO usar herencia cuando**:
  - Subclase rompe contrato de la base
  - Subclase necesita lanzar excepciones nuevas
  - Relación es "tiene-un" (usa composición)
  - Subclase cambia comportamiento fundamental

**Referencia oficial:** [SOLID Principles - LSP](https://en.wikipedia.org/wiki/Liskov_substitution_principle)

## 4. Interface Segregation Principle (ISP)

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Tienes interfaz `DataProcessor` con 10 métodos: `read()`, `write()`, `validate()`, `transform()`, `aggregate()`, `visualize()`, `export_to_s3()`, `send_email()`, `log_metrics()`, `backup()`. Creas `SimpleCSVReader` que solo necesita `read()` pero **debe implementar los 10 métodos** 💥 9 métodos con `raise NotImplementedError` 💥 **Interfaz gorda** 💥

Sin ISP: Interfaces grandes fuerzan implementaciones innecesarias. Clases dependen de métodos que no usan. **Acoplamiento alto** 💥 **Cambios en interfaz afectan a todos** 💥

Con ISP: Interfaces pequeñas y específicas. `Readable`, `Writable`, `Transformable`. `SimpleCSVReader` solo implementa `Readable` ✅ **Solo lo que necesita** ✅

**Ejemplo concreto para juniors**:

Interfaz `Worker` con métodos `work()`, `eat()`, `sleep()`. Robot implementa `Worker` pero **no come ni duerme** 💥 Debe implementar `eat()` y `sleep()` con excepciones 💥

Con ISP: `Workable`, `Eatable`, `Sleepable`. Robot solo implementa `Workable` ✅ Humano implementa los 3 ✅

**Consecuencias de NO usarlo**:
- **Implementaciones vacías** → métodos con `pass` o `raise NotImplementedError`
- **Acoplamiento alto** → clases dependen de métodos que no usan
- **Cambios costosos** → modificar interfaz afecta a todos ($40K)
- **Testing complejo** → mockear métodos innecesarios
- **Confusión** → no está claro qué métodos son realmente necesarios

### 📚 El Concepto

**Definición técnica**:

**Interface Segregation Principle (ISP)**: Los clientes no deben ser forzados a depender de interfaces que no usan. Es mejor tener **muchas interfaces específicas** que una interfaz general grande. Divide interfaces grandes en interfaces más pequeñas y cohesivas.

**Cómo funciona internamente**:
1. Identificas interfaz grande con muchos métodos
2. Agrupas métodos por responsabilidad/uso
3. Creas interfaces pequeñas para cada grupo
4. Clases implementan solo las interfaces que necesitan
5. Clientes dependen solo de interfaces específicas
6. Cambios en una interfaz no afectan otras

**Terminología clave**:
- **Interface segregation**: Dividir interfaces grandes en pequeñas
- **Fat interface**: Interfaz con muchos métodos (anti-pattern)
- **Role interface**: Interfaz pequeña que representa un rol específico
- **Client-specific interface**: Interfaz diseñada para necesidades específicas del cliente
- **Cohesión**: Qué tan relacionados están los métodos de una interfaz

### ❌ Ejemplo Incorrecto: Interfaz Gorda

**Código**:

In [ ]:
# ❌ BAD: Fat interface - forces unnecessary implementations
from abc import ABC, abstractmethod
from typing import List, Dict, Any

class DataProcessor(ABC):
    """Fat interface - TOO MANY methods! Violates ISP."""
    
    @abstractmethod
    def read(self, path: str) -> List[Dict]:
        """Read data."""
        pass
    
    @abstractmethod
    def write(self, data: List[Dict], path: str) -> None:
        """Write data."""
        pass
    
    @abstractmethod
    def validate(self, data: List[Dict]) -> bool:
        """Validate data."""
        pass
    
    @abstractmethod
    def transform(self, data: List[Dict]) -> List[Dict]:
        """Transform data."""
        pass
    
    @abstractmethod
    def aggregate(self, data: List[Dict]) -> Dict:
        """Aggregate data."""
        pass

class SimpleCSVReader(DataProcessor):
    """Simple CSV reader - only needs read()! But must implement ALL methods! 💥"""
    
    def read(self, path: str) -> List[Dict]:
        """Read CSV - the ONLY method we need!"""
        print(f"Reading CSV from {path}")
        return [{"id": 1, "value": "test"}]
    
    # Must implement these even though we don't need them! 💥
    def write(self, data: List[Dict], path: str) -> None:
        raise NotImplementedError("SimpleCSVReader doesn't write!")
    
    def validate(self, data: List[Dict]) -> bool:
        raise NotImplementedError("SimpleCSVReader doesn't validate!")
    
    def transform(self, data: List[Dict]) -> List[Dict]:
        raise NotImplementedError("SimpleCSVReader doesn't transform!")
    
    def aggregate(self, data: List[Dict]) -> Dict:
        raise NotImplementedError("SimpleCSVReader doesn't aggregate!")

# Usage - dangerous! Methods throw exceptions!
reader = SimpleCSVReader()
data = reader.read("data.csv")  # ✅ Works

try:
    reader.write(data, "output.csv")  # 💥 Fails!
except NotImplementedError as e:
    print(f"\n💥 ISP violation! {e}")
    print("SimpleCSVReader forced to implement methods it doesn't need!")

**Problemas**:
- **Interfaz gorda** → 5 métodos, solo necesita 1
- **Implementaciones vacías** → 4 métodos con `NotImplementedError`
- **Acoplamiento alto** → depende de métodos que no usa
- **Bugs en runtime** → llamar método no implementado explota
- **Confusión** → no está claro qué métodos son realmente funcionales
- **Testing complejo** → mockear métodos innecesarios

### ✅ Ejemplo Correcto: Interfaces Segregadas

**Código**:

In [ ]:
# ✅ GOOD: Segregated interfaces - implement only what you need
from abc import ABC, abstractmethod
from typing import List, Dict, Any

class Readable(ABC):
    """Interface for reading data - SMALL and FOCUSED."""
    
    @abstractmethod
    def read(self, path: str) -> List[Dict]:
        """
        Read data from path.
        
        :param path: Path to read from
        :type path: str
        :return: Data records
        :rtype: List[Dict]
        """
        pass

class Writable(ABC):
    """Interface for writing data - SMALL and FOCUSED."""
    
    @abstractmethod
    def write(self, data: List[Dict], path: str) -> None:
        """
        Write data to path.
        
        :param data: Data to write
        :type data: List[Dict]
        :param path: Path to write to
        :type path: str
        """
        pass

class Transformable(ABC):
    """Interface for transforming data - SMALL and FOCUSED."""
    
    @abstractmethod
    def transform(self, data: List[Dict]) -> List[Dict]:
        """
        Transform data.
        
        :param data: Data to transform
        :type data: List[Dict]
        :return: Transformed data
        :rtype: List[Dict]
        """
        pass

class Validatable(ABC):
    """Interface for validating data - SMALL and FOCUSED."""
    
    @abstractmethod
    def validate(self, data: List[Dict]) -> bool:
        """
        Validate data.
        
        :param data: Data to validate
        :type data: List[Dict]
        :return: True if valid
        :rtype: bool
        """
        pass

class SimpleCSVReader(Readable):
    """Simple CSV reader - implements ONLY Readable! ✅"""
    
    def read(self, path: str) -> List[Dict]:
        """Read CSV - the ONLY method we need!"""
        print(f"Reading CSV from {path}")
        return [{"id": 1, "value": "test"}]

class CSVWriter(Writable):
    """CSV writer - implements ONLY Writable! ✅"""
    
    def write(self, data: List[Dict], path: str) -> None:
        """Write CSV."""
        print(f"Writing CSV to {path}")

class DataTransformer(Transformable):
    """Data transformer - implements ONLY Transformable! ✅"""
    
    def transform(self, data: List[Dict]) -> List[Dict]:
        """Transform data."""
        print("Transforming data...")
        return [{**d, "transformed": True} for d in data]

class FullProcessor(Readable, Writable, Transformable, Validatable):
    """Full processor - implements ALL interfaces when needed! ✅"""
    
    def read(self, path: str) -> List[Dict]:
        """Read data."""
        print(f"Reading from {path}")
        return [{"id": 1}]
    
    def write(self, data: List[Dict], path: str) -> None:
        """Write data."""
        print(f"Writing to {path}")
    
    def transform(self, data: List[Dict]) -> List[Dict]:
        """Transform data."""
        return [{**d, "processed": True} for d in data]
    
    def validate(self, data: List[Dict]) -> bool:
        """Validate data."""
        return len(data) > 0

# Usage - each class implements only what it needs!
reader = SimpleCSVReader()
data = reader.read("data.csv")  # ✅ Works

writer = CSVWriter()
writer.write(data, "output.csv")  # ✅ Works

transformer = DataTransformer()
transformed = transformer.transform(data)  # ✅ Works

processor = FullProcessor()
processor.read("input.csv")  # ✅ Works
processor.write(data, "output.csv")  # ✅ Works

print("\n✅ Each class implements only what it needs! ISP respected!")

**Ventajas**:
- **Interfaces pequeñas** → cada interfaz tiene 1 método
- **Solo lo necesario** → clases implementan solo lo que usan
- **Sin implementaciones vacías** → no hay `NotImplementedError`
- **Acoplamiento bajo** → clases dependen solo de lo que necesitan
- **Cambios aislados** → modificar `Writable` no afecta `Readable`
- **Testing simple** → mockear solo interfaces necesarias

### 📊 Comparación Lado a Lado

| Aspecto | Sin ISP | Con ISP |
|---------|---------|----------|
| Métodos por interfaz | 5+ | 1-2 |
| Implementaciones vacías | Muchas | Ninguna |
| Acoplamiento | Alto | Bajo |
| Cambios | Afectan a todos | Aislados |
| Testing | Complejo | Simple |
| Claridad | Baja | Alta |

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Interfaces pequeñas** → 1-3 métodos relacionados
2. **Solo lo necesario** → clases implementan solo lo que usan
3. **Alta cohesión** → métodos de interfaz están relacionados
4. **Bajo acoplamiento** → clientes dependen solo de lo necesario
5. **Composición** → combinar interfaces pequeñas cuando necesario

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Esta clase implementa métodos que no usa?"
  - SÍ → viola ISP, segregar interfaz ❌
  - NO → cumple ISP ✅

- **Pregúntate**: "¿Tengo métodos con `NotImplementedError` o `pass`?"
  - SÍ → viola ISP, interfaz muy grande ❌
  - NO → cumple ISP ✅

- **Pregúntate**: "¿Puedo dividir esta interfaz en interfaces más pequeñas?"
  - SÍ → hazlo, mejora ISP ✅
  - NO (métodos muy relacionados) → interfaz está bien ✅

**Cuándo usar / NO usar**:
- ✅ **Segregar interfaces cuando**:
  - Interfaz tiene muchos métodos (5+)
  - Clases implementan solo algunos métodos
  - Métodos no están relacionados
  - Diferentes clientes usan diferentes subconjuntos
- ❌ **NO segregar cuando**:
  - Interfaz tiene 1-3 métodos muy relacionados
  - Todas las clases usan todos los métodos
  - Segregar añade complejidad innecesaria

**Referencia oficial:** [SOLID Principles - ISP](https://en.wikipedia.org/wiki/Interface_segregation_principle)

## 5. Dependency Inversion Principle (DIP)

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Tienes `ModelTrainer` que crea directamente `S3Storage()` dentro del código. Cambiar a `GCSStorage` = **modificar `ModelTrainer`** 💥 Testing = **imposible mockear S3** 💥 **Acoplamiento alto** 💥 Código depende de implementación concreta 💥

Sin DIP: Módulos de alto nivel dependen de módulos de bajo nivel. `ModelTrainer` → `S3Storage` (concreto). **Imposible cambiar** 💥 **Imposible testear** 💥

Con DIP: Ambos dependen de abstracción. `ModelTrainer` → `Storage` (interfaz) ← `S3Storage`. **Fácil cambiar** ✅ **Fácil testear** ✅ **Inyección de dependencias** ✅

**Ejemplo concreto para juniors**:

Clase `NotificationService` que crea `EmailSender()` directamente. Cambiar a SMS = **modificar `NotificationService`** 💥 Testing = **enviar emails reales** 💥

Con DIP: `NotificationService` recibe `MessageSender` (interfaz). Inyectas `EmailSender` o `SMSSender`. Testing = **inyectar mock** ✅

**Consecuencias de NO usarlo**:
- **Acoplamiento alto** → código depende de implementaciones concretas ($50K)
- **Testing imposible** → no puedes mockear dependencias
- **Cambios costosos** → cambiar implementación = modificar múltiples clases
- **Reutilización imposible** → no puedes usar clase con otra implementación
- **Rigidez** → código inflexible, difícil de extender

### 📚 El Concepto

**Definición técnica**:

**Dependency Inversion Principle (DIP)**: 
1. **Módulos de alto nivel no deben depender de módulos de bajo nivel**. Ambos deben depender de abstracciones.
2. **Abstracciones no deben depender de detalles**. Detalles deben depender de abstracciones.

En otras palabras: **Depende de interfaces, no de implementaciones concretas**.

**Cómo funciona internamente**:
1. Identificas dependencias concretas en código de alto nivel
2. Creas abstracción (interfaz/ABC) para la dependencia
3. Código de alto nivel depende de la abstracción
4. Implementaciones concretas implementan la abstracción
5. Inyectas implementación concreta desde fuera (Dependency Injection)
6. Código de alto nivel no conoce implementación concreta

**Terminología clave**:
- **High-level module**: Módulo con lógica de negocio (ModelTrainer)
- **Low-level module**: Módulo con detalles de implementación (S3Storage)
- **Abstraction**: Interfaz o ABC que define contrato
- **Dependency Injection**: Pasar dependencias desde fuera
- **Inversion of Control (IoC)**: Framework controla flujo, no tu código

### ❌ Ejemplo Incorrecto: Dependencia de Concretos

**Código**:

In [ ]:
# ❌ BAD: High-level depends on low-level concrete class
from typing import Any

class S3Storage:
    """Concrete S3 storage implementation."""
    
    def save(self, model: Any, path: str) -> str:
        """Save to S3."""
        print(f"Saving to S3: {path}")
        return f"s3://{path}"

class ModelTrainer:
    """Model trainer - VIOLATES DIP! Depends on concrete S3Storage."""
    
    def __init__(self):
        # Creates concrete dependency directly! 💥
        self.storage = S3Storage()  # Tightly coupled to S3!
    
    def train_and_save(self, X: list, y: list, model_id: str) -> str:
        """
        Train and save model.
        PROBLEM: Cannot change storage, cannot test without S3!
        """
        print("Training model...")
        model = "trained_model"
        
        # Uses concrete S3Storage - cannot swap! 💥
        url = self.storage.save(model, model_id)
        return url

# Usage - tightly coupled, hard to test
trainer = ModelTrainer()  # Always uses S3! Cannot change!
url = trainer.train_and_save([1, 2], [0, 1], "model_v1")
print(f"Model saved at: {url}")

print("\n💥 Problems:")
print("- Cannot change to GCS without modifying ModelTrainer")
print("- Cannot test without real S3 connection")
print("- High-level (ModelTrainer) depends on low-level (S3Storage)")

**Problemas**:
- **Acoplamiento alto** → `ModelTrainer` depende de `S3Storage` concreto
- **Imposible cambiar** → cambiar a GCS = modificar `ModelTrainer`
- **Testing imposible** → no puedes mockear `S3Storage`
- **Reutilización imposible** → no puedes usar `ModelTrainer` con otro storage
- **Violación DIP** → alto nivel depende de bajo nivel
- **Rigidez** → código inflexible

### ✅ Ejemplo Correcto: Dependencia de Abstracciones

**Código**:

In [ ]:
# ✅ GOOD: Both depend on abstraction (DIP)
from abc import ABC, abstractmethod
from typing import Any

class Storage(ABC):
    """Abstract storage interface - ABSTRACTION that both depend on."""
    
    @abstractmethod
    def save(self, model: Any, path: str) -> str:
        """
        Save model to storage.
        
        :param model: Model to save
        :type model: Any
        :param path: Storage path
        :type path: str
        :return: Storage URL
        :rtype: str
        """
        pass

class S3Storage(Storage):
    """S3 storage implementation - depends on Storage abstraction."""
    
    def save(self, model: Any, path: str) -> str:
        """Save to S3."""
        print(f"Saving to S3: {path}")
        return f"s3://{path}"

class GCSStorage(Storage):
    """GCS storage implementation - depends on Storage abstraction."""
    
    def save(self, model: Any, path: str) -> str:
        """Save to GCS."""
        print(f"Saving to GCS: {path}")
        return f"gs://{path}"

class LocalStorage(Storage):
    """Local storage implementation - depends on Storage abstraction."""
    
    def save(self, model: Any, path: str) -> str:
        """Save to local filesystem."""
        print(f"Saving to local: {path}")
        return f"file://{path}"

class ModelTrainer:
    """Model trainer - FOLLOWS DIP! Depends on Storage abstraction."""
    
    def __init__(self, storage: Storage) -> None:
        """
        Initialize trainer with storage.
        
        :param storage: Storage implementation (injected!)
        :type storage: Storage
        """
        # Depends on abstraction, not concrete class! ✅
        self.storage = storage
    
    def train_and_save(self, X: list, y: list, model_id: str) -> str:
        """
        Train and save model.
        
        :param X: Training features
        :type X: list
        :param y: Training labels
        :type y: list
        :param model_id: Model identifier
        :type model_id: str
        :return: Storage URL
        :rtype: str
        """
        print("Training model...")
        model = "trained_model"
        
        # Uses abstraction - works with ANY Storage! ✅
        url = self.storage.save(model, model_id)
        return url

# Usage - loosely coupled, easy to change and test!
print("Using S3 Storage:")
s3_trainer = ModelTrainer(S3Storage())  # Inject S3
url1 = s3_trainer.train_and_save([1, 2], [0, 1], "model_v1")
print(f"Saved at: {url1}\n")

print("Using GCS Storage:")
gcs_trainer = ModelTrainer(GCSStorage())  # Inject GCS - no code change!
url2 = gcs_trainer.train_and_save([1, 2], [0, 1], "model_v2")
print(f"Saved at: {url2}\n")

print("Using Local Storage:")
local_trainer = ModelTrainer(LocalStorage())  # Inject Local - no code change!
url3 = local_trainer.train_and_save([1, 2], [0, 1], "model_v3")
print(f"Saved at: {url3}\n")

# Testing - inject mock!
class MockStorage(Storage):
    """Mock storage for testing."""
    def save(self, model: Any, path: str) -> str:
        return f"mock://{path}"

print("Testing with Mock:")
test_trainer = ModelTrainer(MockStorage())  # Inject mock - easy testing!
url4 = test_trainer.train_and_save([1, 2], [0, 1], "model_test")
print(f"Saved at: {url4}")

print("\n✅ Benefits:")
print("- Easy to change storage (just inject different implementation)")
print("- Easy to test (inject mock)")
print("- High-level (ModelTrainer) depends on abstraction (Storage)")
print("- Low-level (S3Storage, GCSStorage) depends on abstraction (Storage)")

**Ventajas**:
- **Bajo acoplamiento** → `ModelTrainer` depende de abstracción
- **Fácil cambiar** → inyectar diferente storage, sin modificar código
- **Testing simple** → inyectar mock, sin dependencias reales
- **Reutilización** → usar `ModelTrainer` con cualquier storage
- **Cumple DIP** → ambos niveles dependen de abstracción
- **Flexibilidad** → código extensible y mantenible

### 📊 Comparación Lado a Lado

| Aspecto | Sin DIP | Con DIP |
|---------|---------|----------|
| Dependencia | Concreto | Abstracción |
| Acoplamiento | Alto | Bajo |
| Cambiar implementación | Modificar código | Inyectar diferente |
| Testing | Imposible | Fácil (mock) |
| Reutilización | Imposible | Fácil |
| Flexibilidad | Rígido | Flexible |

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Depende de abstracciones** → no de implementaciones concretas
2. **Inyección de dependencias** → pasar dependencias desde fuera
3. **Ambos niveles dependen de abstracción** → alto y bajo nivel
4. **Testing fácil** → inyectar mocks en lugar de dependencias reales
5. **Flexibilidad** → cambiar implementación sin modificar código

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Esta clase crea sus dependencias con `new`/`()`?"
  - SÍ → viola DIP, usar inyección ❌
  - NO (recibe por constructor) → cumple DIP ✅

- **Pregúntate**: "¿Puedo testear esta clase sin dependencias reales?"
  - NO → viola DIP, inyectar dependencias ❌
  - SÍ (inyecto mocks) → cumple DIP ✅

- **Pregúntate**: "¿Puedo cambiar implementación sin modificar esta clase?"
  - NO → viola DIP, depender de abstracción ❌
  - SÍ (inyecto diferente) → cumple DIP ✅

**Cuándo usar / NO usar**:
- ✅ **Usar DIP cuando**:
  - Clase tiene dependencias externas (DB, API, storage)
  - Necesitas testear sin dependencias reales
  - Quieres cambiar implementación fácilmente
  - Múltiples implementaciones posibles
- ❌ **NO sobre-aplicar cuando**:
  - Dependencia es simple y estable (ej: `datetime`)
  - Solo una implementación posible
  - Abstracción añade complejidad innecesaria

**Patrones relacionados**:
- **Dependency Injection**: Pasar dependencias por constructor
- **Factory Pattern**: Crear objetos sin especificar clase exacta
- **Strategy Pattern**: Intercambiar algoritmos en runtime
- **Repository Pattern**: Abstraer acceso a datos

**Referencia oficial:** [SOLID Principles - DIP](https://en.wikipedia.org/wiki/Dependency_inversion_principle)

## Resumen

1. **Single Responsibility Principle (SRP)** → Una clase = una responsabilidad = una razón para cambiar. Divide clases grandes en clases pequeñas y cohesivas. Beneficio: testing simple, bajo acoplamiento, alta reutilización.

2. **Open/Closed Principle (OCP)** → Abierto para extensión, cerrado para modificación. Añade funcionalidad creando nuevas clases, no modificando existentes. Beneficio: cero riesgo en código estable, extensibilidad sin bugs.

3. **Liskov Substitution Principle (LSP)** → Subclases deben poder sustituir clase base sin romper comportamiento. Respeta contratos, no lances excepciones nuevas. Beneficio: polimorfismo funciona, código predecible.

4. **Interface Segregation Principle (ISP)** → Muchas interfaces específicas > una interfaz gorda. Clientes no deben depender de métodos que no usan. Beneficio: sin implementaciones vacías, bajo acoplamiento.

5. **Dependency Inversion Principle (DIP)** → Depende de abstracciones, no de concretos. Alto y bajo nivel dependen de interfaces. Inyecta dependencias. Beneficio: testing fácil, flexibilidad, bajo acoplamiento.

**Todos juntos**: SOLID crea código **mantenible**, **testeable**, **extensible**, y **flexible**. Crítico para proyectos de ML/Data que escalan.

## Preguntas de Autoevaluación

### 1. ¿Cómo sabes si una clase viola el Single Responsibility Principle?

**Respuesta:** Si la clase tiene más de una razón para cambiar, viola SRP. Pregúntate: "¿Puedo describir esta clase en una frase sin usar 'y'?" Si necesitas decir "hace X y Y y Z", viola SRP. También, si necesitas mockear múltiples dependencias para testear, probablemente viola SRP.

### 2. ¿Cuál es la diferencia entre Open/Closed Principle y simplemente añadir código nuevo?

**Respuesta:** OCP significa que puedes añadir funcionalidad **sin modificar código existente**. No es solo añadir código, es añadir código **sin tocar** lo que ya funciona. Usas abstracción (interfaces/ABC) para que el código existente funcione con nuevas implementaciones sin cambios.

### 3. ¿Por qué `PretrainedModel` que lanza excepción en `train()` viola Liskov Substitution Principle?

**Respuesta:** Porque rompe el contrato de la clase base `Model`, que dice que `train()` debe funcionar con datos válidos. Código que funciona con `Model` espera que `train()` no lance excepciones, pero `PretrainedModel` lo hace. Esto significa que `PretrainedModel` no puede sustituir `Model` sin romper el programa. La solución correcta es hacer `train()` un no-op (no hacer nada) en lugar de lanzar excepción.

### 4. ¿Cuándo deberías dividir una interfaz grande en interfaces más pequeñas (ISP)?

**Respuesta:** Cuando clases implementan la interfaz pero tienen métodos con `NotImplementedError` o `pass`, o cuando diferentes clientes usan diferentes subconjuntos de métodos. Si una interfaz tiene 5+ métodos no relacionados, probablemente viola ISP. Divide cuando los métodos representan responsabilidades diferentes que no todas las implementaciones necesitan.

### 5. ¿Cómo se relacionan Dependency Inversion Principle y Dependency Injection?

**Respuesta:** DIP es el **principio** (depende de abstracciones, no concretos). Dependency Injection es la **técnica** para implementar DIP (pasar dependencias por constructor en lugar de crearlas internamente). DI permite que el código de alto nivel dependa de abstracciones mientras que las implementaciones concretas se inyectan desde fuera, cumpliendo así con DIP.

### 6. ¿Cuál es el principio SOLID más importante para testing?

**Respuesta:** **Dependency Inversion Principle (DIP)** es el más crítico para testing porque permite inyectar mocks en lugar de dependencias reales. Sin embargo, **Single Responsibility Principle (SRP)** también es crucial porque hace que las clases sean más fáciles de testear al tener menos responsabilidades. En la práctica, necesitas ambos: SRP para clases pequeñas y testeables, DIP para poder mockear dependencias.

### 7. ¿Puedes sobre-aplicar SOLID? ¿Cuándo es demasiado?

**Respuesta:** Sí, puedes sobre-aplicar SOLID. Es demasiado cuando:
- Creas abstracciones para casos simples que nunca cambiarán (YAGNI - You Aren't Gonna Need It)
- Divides clases tan pequeñas que pierdes cohesión y añades complejidad
- Creas interfaces para cada clase "por si acaso"
- El código se vuelve más difícil de entender por exceso de indirección

Aplica SOLID cuando resuelve un problema real, no "por las buenas prácticas". Empieza simple, refactoriza cuando necesites la flexibilidad.

## Recursos y Referencias Oficiales

### Documentación Oficial
- **[Python Classes Tutorial](https://docs.python.org/3/tutorial/classes.html)**: Tutorial oficial sobre clases en Python
  - Fundamentos de OOP en Python
  - Herencia y polimorfismo

- **[Python ABC Module](https://docs.python.org/3/library/abc.html)**: Documentación del módulo abc
  - Abstract Base Classes para definir interfaces
  - Decorators para métodos abstractos

### Estándares/PEPs
- **[PEP 3119 - Abstract Base Classes](https://peps.python.org/pep-3119/)**: PEP que introduce ABC
  - Motivación y diseño de ABC en Python
  - Cómo usar ABC para definir interfaces

### Mejores Prácticas
- **[SOLID Principles in Python](https://realpython.com/solid-principles-python/)**: Guía completa de Real Python
  - Explicación detallada de cada principio
  - Ejemplos prácticos en Python
  - Cuándo aplicar cada principio

- **[Clean Code in Python](https://realpython.com/python-clean-code/)**: Principios de código limpio
  - SOLID en contexto de código limpio
  - Refactoring patterns

### Libros Recomendados
- **Clean Code by Robert C. Martin**: El libro original sobre principios SOLID
  - Definición autoritativa de SOLID
  - Ejemplos en Java pero conceptos aplicables a Python

- **Clean Architecture by Robert C. Martin**: Arquitectura basada en SOLID
  - Cómo aplicar SOLID a nivel de sistema
  - Dependency Inversion en arquitectura

### Artículos y Recursos
- **[SOLID Principles - Wikipedia](https://en.wikipedia.org/wiki/SOLID)**: Resumen enciclopédico
  - Historia y contexto de SOLID
  - Enlaces a cada principio individual

- **[Dependency Injection in Python](https://python-dependency-injector.ets-labs.org/)**: Framework de DI
  - Cómo implementar DI en Python
  - Patrones avanzados de inyección

### Herramientas Relacionadas
- **[typing.Protocol](https://docs.python.org/3/library/typing.html#typing.Protocol)**: Structural subtyping
  - Alternativa a ABC para interfaces
  - Duck typing con type hints

- **[mypy](http://mypy-lang.org/)**: Type checker para Python
  - Verifica type hints en tiempo de desarrollo
  - Detecta violaciones de contratos

### Notas Importantes
- Todos los enlaces están actualizados a partir de 2025
- Se recomienda revisar la documentación oficial regularmente
- SOLID son principios, no reglas absolutas - usa juicio para aplicarlos
- En Python, duck typing y Protocols pueden ser alternativas a ABC en algunos casos